In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv("/scratch/arjunth2001/output21.csv")

In [ ]:
import json
import pandas as pd
with open("../data/dev-v2.0.json") as f:
    data = json.load(f)
data = data['data']


In [ ]:
from tqdm.auto import tqdm
_ans = []
corpus = []
for doc in tqdm(data):
    title = doc['title']
    paragraphs = doc['paragraphs']
    for para in paragraphs:
        qas = para['qas']
        for qa in qas:
            question = qa['question']
            context = para['context']
            answers = [a['text'] for a in qa['answers']]
            if (question, answers, context, title) not in corpus:
                corpus.append((question, answers,context,title))
                _ans.append(answers)


In [ ]:
list(set(_ans[0]))

In [ ]:
tqdm.pandas()

In [ ]:
df.head()

In [ ]:
import spacy
nlp = spacy.load("en_core_web_sm")
truths = []
pos =[]
ner = []
for i in tqdm(range(len(df))):
    truths.append(_ans[df['q_id'][i]])
    doc = nlp(str(df["answer"][i]))
    pos.append([token.tag_ for token in doc])
    ner.append([token.ent_type_ for token in doc])
df["truths"] = truths
df["pos"] = pos
df["ner"] = ner


In [ ]:
targets = []
for i in tqdm(range(len(df))):
    if df["truths"][i] == []:
        targets.append(0)
    elif str(df["answer"][i]).lower() in " ".join([str(e).lower() for e in df["truths"][i]]):
        targets.append(1)
    else:
        targets.append(0)


In [ ]:
df["target"] = targets

In [ ]:
df = df[df.truths.str.len()>= 1]

In [ ]:
len(df["q_id"].unique())

In [ ]:
df.reset_index(inplace=True)

In [ ]:
import numpy as np

In [ ]:
doc_sim = df.groupby("doc_id").agg(
    sum_doc_score=pd.NamedAgg(column="par_sim", aggfunc="sum"),
    max_doc_score=pd.NamedAgg(column="par_sim", aggfunc="max"),
    min_doc_score=pd.NamedAgg(column="par_sim", aggfunc="min"),
    avg_doc_score=pd.NamedAgg(column="par_sim", aggfunc=np.mean),
)

In [ ]:
doc_sim

In [ ]:
span_sim = df.groupby("answer").agg(
    sum_span_score=pd.NamedAgg(column="score", aggfunc="sum"),
    max_span_score=pd.NamedAgg(column="score", aggfunc="max"),
    min_span_score=pd.NamedAgg(column="score", aggfunc="min"),
    avg_span_score=pd.NamedAgg(column="score", aggfunc=np.mean),
)

In [ ]:
span_sim["sum_span_score"][148]

In [ ]:
sss = []
mass = []
misc = []
ass = []
sds =[]
mads =[]
mids = []
ads = []
for i in tqdm(range(len(df))):
    try:
        sss.append(span_sim["sum_span_score"][df["answer"][i]])
        mass.append(span_sim["max_span_score"][df["answer"][i]])
        misc.append(span_sim["min_span_score"][df["answer"][i]])
        ass.append(span_sim["avg_span_score"][df["answer"][i]])
        sds.append(doc_sim["sum_doc_score"][df["doc_id"][i]])
        mads.append(doc_sim["max_doc_score"][df["doc_id"][i]])
        mids.append(doc_sim["min_doc_score"][df["doc_id"][i]])
        ads.append(doc_sim["avg_doc_score"][df["doc_id"][i]])
    except:
        sss.append(0)
        mass.append(0)
        misc.append(0)
        ass.append(0)
        sds.append(0)
        mads.append(0)
        mids.append(0)
        ads.append(0)
        print("here")

In [ ]:
df["sum_span_score"] = sss
df["max_span_score"] = mass
df["min_span_score"] = misc
df["avg_span_score"] = ass
df["sum_doc_score"] = sds
df["max_doc_score"] = mads
df["min_doc_score"] = mids
df["avg_doc_score"] = ads

In [ ]:
qids = list(df["q_id"].unique())

In [ ]:
sizes = []
tables = []
for qg in df.groupby("q_id"):
    new = qg[1].drop_duplicates(subset="answer")
    new.sort_values(by=['score'],inplace=True,ascending=False)
    new.reset_index(drop=True,inplace=True)
    new = new.head(4)
    answer_list = qg[1]["answer"].tolist()
    new["first_occurence"] = [0]*len(new)
    new["num_occurence"] = [1]*len(new)
    new.reset_index(inplace=True)
    for i in range(len(new)):
        if new["answer"][i] in answer_list:
            new["first_occurence"][i] = answer_list.index(new["answer"][i])
            new["num_occurence"][i] = answer_list.count(new["answer"][i])
    sizes.append(len(new))
    tables.append(new)
    

In [ ]:
set(sizes)

In [ ]:
tables = [t for t in tables if len(t) == 4]

In [ ]:
t = pd.concat(tables)

In [ ]:
t["target"].value_counts()

In [ ]:
t.to_csv("../data/t2.csv",index=False)